In [2]:
import clr  # pythonnet module
import time
from matplotlib import pyplot as plt
from PIL import Image
import sys
import numpy as np
import ctypes
from System.Runtime.InteropServices import GCHandle, GCHandleType
import itertools
from six import PY2


sys.path.append(r"C:\Program Files (x86)\FLIR Systems\FLIR Atlas SDK 6\bin\x64")  # path of dll
clr.AddReference("Flir.Atlas.Live")
clr.AddReference("FLir.Atlas.Image")

In [3]:

from Flir.Atlas.Live import Device, Discovery
from Flir.Atlas.Image import ThermalImage, Palettes

In [4]:

disc = Discovery.Discovery()

disc_cameras = []


def device_found(sender, event):
    camera_info = event.CameraDevice
    disc_cameras.append(camera_info)
    print(camera_info.Name + " " + camera_info.SerialNumber)


disc.DeviceFound += device_found

# disc.Start(Discovery.Interface.Network)
disc.Start()
disc.Dispose()

for d in disc_cameras:
    if d.Name == "FLIR T1020":
        device = d
print(device.Name)

Camera Emulator Unknown
FLIR T1020 72503013
HP Wide Vision HD Unknown
FLIR T1020


In [8]:
# cam = Device.ThermalCamera()  # Or VideoOverlay
# device.SelectedStreamingFormat = Discovery.ImageFormat.FlirFileFormat  # device.StreamingFormats[0]
# print(device.SelectedStreamingFormat)  # Enumeration 0 or 1. 0 is FLirFileFOrmat (thermal matrix) 1 is RGB
# print("SUpported formatss", device.StreamingFormats)

cam = Device.VideoOverlayCamera()
cam.Connect(device)
cam.StartGrabbing()
# %% grab a single image

print("Connecting...")
while not cam.IsGrabbing:
    time.sleep(.1)
print("Starting Acquisition...")
print("Connection Status", cam.ConnectionStatus)
print("isGrabbing", cam.IsGrabbing)
print("Fps: ", cam.Fps)
print("Device Info:", cam.Diagnostics)
# print("Bla")
# print("Is grabbing", cam.IsGrabbing)
im = cam.GetImage()
print(im)
print(im.Size)

Connecting...
Starting Acquisition...
Connection Status 3
isGrabbing True
Fps:  0.0
Device Info: Flir.Atlas.Live.Device.Diagnostics
Flir.Atlas.Image.VisualImage
{Width=1024, Height=768}


In [9]:
cam.StopGrabbing()
cam.Disconnect()

In [24]:
# im.Palette = Palettes.PaletteManager.Iron
x = im.ImageArray()

# print(x[0])
# plt.imshow(im.Bitmap)
# save the image
# print(im.Bitmap)
#
# path2im = r"C:\Users\Olivier Desclaux\OneDrive\Recherche\PICUPE\FLIR Atlas SDK\image.jpg"
# im.SaveSnapshot(path2im)
# #
# # # show the image
# #
# #
# img = Image.open(path2im)
# plt.imshow(img)

# %% disconnect the camera
cam.StopGrabbing()
cam.Disconnect()

InvalidOperationException: Failed to create bitmap image
   à Flir.Atlas.Image.VisualImage.ImageArray()

In [13]:
_MAP_NET_NP = {
    'Single': np.dtype('float32'),
    'Double': np.dtype('float64'),
    'SByte': np.dtype('int8'),
    'Int16': np.dtype('int16'),
    'Int32': np.dtype('int32'),
    'Int64': np.dtype('int64'),
    'Byte': np.dtype('uint8'),
    'UInt16': np.dtype('uint16'),
    'UInt32': np.dtype('uint32'),
    'UInt64': np.dtype('uint64'),
    'Boolean': np.dtype('bool'),
}

def net_to_numpy(dotnet_array):
    '''
    Given a .NET CLR `System.Array` returns a `numpy.ndarray`.  See _MAP_NET_NP for
    the mapping of CLR types to Numpy dtypes.
    '''
    dims = np.empty(dotnet_array.Rank, dtype=int)
    for I in range(dotnet_array.Rank):
        dims[I] = dotnet_array.GetLength(I)
    netType = dotnet_array.GetType().GetElementType().Name

    if (netType not in _MAP_NET_NP):
        raise NotImplementedError("net_to_numpy does not yet support System type {}".format(netType))

    map_result = _MAP_NET_NP[netType]

    if (isinstance(map_result, tuple)):
        ntype = map_result[0]
        converter = map_result[1]
    else:
        ntype = map_result
        converter = None

    if ntype.kind == 'U':
        # Need to find the size
        size = 0
        pattern = [range(x) for x in dims]
        for t in itertools.product(*pattern):
            # only a list in python2, otherwise a generator
            t_list = t if PY2 else list(t)
            s = len(dotnet_array.GetValue(t_list))
            if s > size:
                size = s
        ntype = np.dtype(('U', size))

    npArray = np.empty(dims, order='C', dtype=ntype)

    try:
        sourceHandle = GCHandle.Alloc(dotnet_array, GCHandleType.Pinned)
        sourcePtr = sourceHandle.AddrOfPinnedObject().ToInt64()
        destPtr = npArray.__array_interface__['data'][0]
        ctypes.memmove(destPtr, sourcePtr, npArray.nbytes)
    finally:
        if sourceHandle.IsAllocated:
            sourceHandle.Free()

    return npArray

In [23]:
im.GetData()